# Demonstrate ADES Execution for OGC Application Packages
## This notebook runs through some example API calls to the ADES (Application, Deployment Execution Service) component of the EODH Platform

In [1]:
!pip install urllib3

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import json
import time
import urllib3
http = urllib3.PoolManager(cert_reqs='CERT_NONE')
urllib3.disable_warnings() ## to avoid SSL warnings

In [3]:
## Define text colour for later output
class bcolors:
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    ENDC = '\033[0m'

In [27]:
## Place your username and password here
username = "username"
password = "password"
auth_dict = urllib3.make_headers(basic_auth=f'{username}:{password}')

## Below are some example API requests you can make to the ADES component
Feel free to run these examples and change the inputs by specifying the application packages, process name and process inputs.
All outputs can be found in the S3 bucket [eodhp-ades](https://s3.console.aws.amazon.com/s3/buckets/eodhp-ades?region=eu-west-2&bucketType=general&tab=objects).

As an example we provide three EOEPCA-developed OGC Application Package to demonstrate the successful execution using the ADES deployment:
- [convert-url](https://github.com/EOEPCA/convert/blob/main/convert-url-app.cwl) - take an image specified by a URL and resize it by a given scale percentage
- [convert-stac](https://github.com/EOEPCA/convert/blob/main/convert-stac-app.cwl) - take an image specified by a stac item and resize it by a given scale percentage
- [water-bodies](https://github.com/EOEPCA/deployment-guide/blob/main/deploy/samples/requests/processing/water-bodies-app.cwl) - takes STAC items, area of interest, epsg definition and set of bands and identifies water bodies based on NDWI and Otsu threshold

This application is specified by configuring the below variable

In [5]:
process_to_be_run = "water-bodies"

In [6]:
# Update these variables as required to identify the running ades instance and specify workspace name
# If the workspace does not yet exect, it will be created by the ades automatically
ades_endpoint = "ades.test.eodhp.eco-ke-staging.com"
user = "test_cluster"

# Automated configuration of CWL script location, process name and inputs
if process_to_be_run == "convert-url":
    process_name = "convert-url"
    cwl_location = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/deploy/samples/requests/processing/convert-url-app.cwl"
    inputs_dict = {"inputs": {
                    "fn": "resize",
                    "stac":  "https://eoepca.org/media_portal/images/logo6_med.original.png",
                    "size": "50%"
                    }
                  }
elif process_to_be_run == "convert-stac":
    process_name = "convert-stac"
    cwl_location = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/deploy/samples/requests/processing/convert-stac-app.cwl"
    inputs_dict = {"inputs": {
                    "fn": "resize",
                    "stac":  "https://raw.githubusercontent.com/EOEPCA/convert/main/stac/eoepca-logo.json",
                    "size": "50%"
                    }
                  }

elif process_to_be_run == "water-bodies":
    process_name = "water-bodies"
    cwl_location = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/deploy/samples/requests/processing/water-bodies-app.cwl"
    inputs_dict = {"inputs": {
                    "stac_items": [
                        "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A",
                        "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A"
                    ],
                    "aoi": "-121.399,39.834,-120.74,40.472",
                    "epsg": "EPSG:4326",
                    "bands": [
                        "green",
                        "nir"
                    ]
                    }
                  }

### List processes

In [7]:
url = f"https://{ades_endpoint}/{user}/ogc-api/processes"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'processes': [{'id': 'display',
   'title': 'Print Cheetah templates as HTML',
   'description': 'Print Cheetah templates as HTML.',
   'mutable': False,
   'version': '2.0.0',
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/json',
     'title': 'Process Description',
     'href': 'https://ades.test.eodhp.eco-ke-staging.com/test_cluster/ogc-api/processes/display'}]},
  {'id': 'echo',
   'title': 'Echo input',
   'description': 'Simply echo the value provided as input',
   'mutable': False,
   'version': '2.0.0',
   'metadata': [{'title': 'Demo'}],
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/json',
     'title': 'Process Description',
     'href': 'https://ades.test.eodhp.eco-ke-staging.com/test_cluster/ogc-api/processes/echo'}]}],
 '

### Undeploy/Delete process

In [8]:
## Here a 204 response means the process was remove successfully, no other content is returned
url = f"https://{ades_endpoint}/{user}/ogc-api/processes/{process_name}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('DELETE', url, headers=headers)
response.status

403

### Deploy processes

In [9]:
url = f"https://{ades_endpoint}/{user}/ogc-api/processes"
headers = {"Accept": "application/json", "Content-Type": "application/json"}
headers.update(auth_dict)
params = {"executionUnit": {
            "href": f"{cwl_location}",
            "type": "application/cwl"
            }
         }
response = http.request('POST', url, headers=headers, body=json.dumps(params))
deployStatus = response.headers['Location']
json.loads(response.data)

{'id': 'water-bodies',
 'title': 'Water bodies detection based on NDWI and otsu threshold',
 'description': 'Water bodies detection based on NDWI and otsu threshold',
 'mutable': True,
 'version': '1.4.1',
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'https://ades.test.eodhp.eco-ke-staging.com/test_cluster/ogc-api/processes/water-bodies/execution'}]}

### Get deploy status

In [10]:
url = f"{deployStatus}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'id': 'water-bodies',
 'title': 'Water bodies detection based on NDWI and otsu threshold',
 'description': 'Water bodies detection based on NDWI and otsu threshold',
 'mutable': True,
 'version': '1.4.1',
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'https://ades.test.eodhp.eco-ke-staging.com/test_cluster/ogc-api/processes/water-bodies/execution'}],
 'inputs': {'aoi': {'title': 'area of interest',
   'description': 'area of interest as a bounding box',
   'schema': {'type': 'string'}},
  'bands': {'title': 'bands used for the NDWI',
   'description': 'bands used for the NDWI',
   'maxOccurs': 999,
   'schema': {'type': 'string', 'default': "['green', 'nir']"}},
  'epsg': {'title': 'EPSG code',
   'description': 'EPSG code',
   'schema': {'type': 'string', 'default': 'EPSG:4326'}},
  'stac_i

### Get process details

In [11]:
url = f"https://{ades_endpoint}/{user}/ogc-api/processes/{process_name}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'id': 'water-bodies',
 'title': 'Water bodies detection based on NDWI and otsu threshold',
 'description': 'Water bodies detection based on NDWI and otsu threshold',
 'mutable': True,
 'version': '1.4.1',
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'https://ades.test.eodhp.eco-ke-staging.com/test_cluster/ogc-api/processes/water-bodies/execution'}],
 'inputs': {'aoi': {'title': 'area of interest',
   'description': 'area of interest as a bounding box',
   'schema': {'type': 'string'}},
  'bands': {'title': 'bands used for the NDWI',
   'description': 'bands used for the NDWI',
   'maxOccurs': 999,
   'schema': {'type': 'string', 'default': "['green', 'nir']"}},
  'epsg': {'title': 'EPSG code',
   'description': 'EPSG code',
   'schema': {'type': 'string', 'default': 'EPSG:4326'}},
  'stac_i

### Execute process

In [12]:
url = f"https://{ades_endpoint}/{user}/ogc-api/processes/{process_name}/execution"
headers = {"Accept": "application/json", "Content-Type": "application/json", "Prefer": "respond-async"}
headers.update(auth_dict)
params = {**inputs_dict}
print(json.dumps(params))
time.sleep(5)
response = http.request('POST', url, headers=headers, body=json.dumps(params))
executeStatus = response.headers['Location']
json.loads(response.data)

{"inputs": {"stac_items": ["https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A", "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A"], "aoi": "-121.399,39.834,-120.74,40.472", "epsg": "EPSG:4326", "bands": ["green", "nir"]}}


{'jobID': '03859658-e5d5-11ee-bb53-128be50dffcf',
 'type': 'process',
 'processID': 'water-bodies',
 'created': '2024-03-19T09:42:34.766Z',
 'started': '2024-03-19T09:42:34.766Z',
 'updated': '2024-03-19T09:42:34.766Z',
 'status': 'running',
 'message': 'ZOO-Kernel accepted to run your service!',
 'links': [{'title': 'Status location',
   'rel': 'status',
   'type': 'application/json',
   'href': 'https://ades.test.eodhp.eco-ke-staging.com/test_cluster/ogc-api/jobs/03859658-e5d5-11ee-bb53-128be50dffcf'}]}

### Get execute status
See the following section to continually poll this function instead to determine once complete

In [13]:
url = f"{executeStatus}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
time.sleep(5)
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'progress': 10,
 'jobID': '03859658-e5d5-11ee-bb53-128be50dffcf',
 'type': 'process',
 'processID': 'water-bodies',
 'created': '2024-03-19T09:42:34.766Z',
 'started': '2024-03-19T09:42:34.766Z',
 'updated': '2024-03-19T09:42:36.303Z',
 'status': 'running',
 'message': 'workflow wrapped, creating processing environment',
 'links': [{'title': 'Status location',
   'rel': 'status',
   'type': 'application/json',
   'href': 'https://ades.test.eodhp.eco-ke-staging.com/test_cluster/ogc-api/jobs/03859658-e5d5-11ee-bb53-128be50dffcf'}]}

### Get execute status (continuous polling)
Run this cell to keep polling the ExecuteStatus endpoint to determine when the process has finished running and also see it's final status: *SUCCESS* or *FAILED*

In [14]:
url = f"{executeStatus}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
data = json.loads(response.data)
status = data['status']
message = data['message']
print("Status is " + bcolors.OKBLUE + status.upper() + bcolors.ENDC)
print("Message is " + "\033[1m" + message + "\033[0m", end="")
old_message = message
old_status = status
while status == "running":
    time.sleep(2)
    response = http.request('GET', url, headers=headers)
    data = json.loads(response.data)
    status = data['status']
    message = data['message']
    if status != old_status:
        print("\n")
        print("Status is " + bcolors.OKBLUE + status.upper() + bcolors.ENDC)
        print("Message is " + "\033[1m" + message + "\033[0m", end="")
    elif message != old_message:
        print(".")
        print("Message is " + "\033[1m" + message + "\033[0m", end="")
    else:
        print(".", end="")
    old_message = message
    old_status = status

if status == "successful":
    print("\n")
    print(bcolors.OKGREEN + "SUCCESS" + bcolors.ENDC)

if status == "failed":
    print(bcolors.WARNING + "FAILED" + bcolors.ENDC)

Status is RUNNING
Message is workflow wrapped, creating processing environment...........
Message is processing environment created, preparing execution......
Message is execution submitted...............................................
Message is delivering outputs, logs and usage report.....

Status is SUCCESSFUL
Message is ZOO-Kernel successfully run your service!

SUCCESS


### Get processing results

In [15]:
## Note, this will return a 500 response when no output is produced
url = f"{executeStatus}/results"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'type': 'Collection',
 'id': '03859658-e5d5-11ee-bb53-128be50dffcf',
 'stac_version': '1.0.0',
 'description': 'description',
 'links': [{'rel': 'root',
   'href': 's3://eodhp-ades/processing-results/03859658-e5d5-11ee-bb53-128be50dffcf/catalog.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': 's3://eodhp-ades/processing-results/03859658-e5d5-11ee-bb53-128be50dffcf/03859658-e5d5-11ee-bb53-128be50dffcf/S2B_10TFK_20210713_0_L2A/S2B_10TFK_20210713_0_L2A.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': 's3://eodhp-ades/processing-results/03859658-e5d5-11ee-bb53-128be50dffcf/03859658-e5d5-11ee-bb53-128be50dffcf/S2A_10TFK_20220524_0_L2A/S2A_10TFK_20220524_0_L2A.json',
   'type': 'application/json'},
  {'rel': 'self',
   'href': 's3://eodhp-ades/processing-results/03859658-e5d5-11ee-bb53-128be50dffcf/03859658-e5d5-11ee-bb53-128be50dffcf/collection.json',
   'type': 'application/json'},
  {'rel': 'parent',
   'href': 's3://eodhp-ades/processing-results/038

### List jobs

In [16]:
url = f"https://{ades_endpoint}/{user}/ogc-api/jobs"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'jobs': [{'jobID': 'e2f8fdce-e5d1-11ee-8f27-128be50dffcf',
   'type': 'process',
   'processID': 'water-bodies',
   'created': '2024-03-19T09:20:11.731Z',
   'started': '2024-03-19T09:20:11.731Z',
   'finished': '2024-03-19T09:23:15.365Z',
   'updated': '2024-03-19T09:23:14.898Z',
   'status': 'successful',
   'message': 'ZOO-Kernel successfully run your service!',
   'links': [{'title': 'Status location',
     'rel': 'status',
     'type': 'application/json',
     'href': 'https://ades.test.eodhp.eco-ke-staging.com/test_cluster/ogc-api/jobs/e2f8fdce-e5d1-11ee-8f27-128be50dffcf'},
    {'title': 'Result location',
     'rel': 'http://www.opengis.net/def/rel/ogc/1.0/results',
     'type': 'application/json',
     'href': 'https://ades.test.eodhp.eco-ke-staging.com/test_cluster/ogc-api/jobs/e2f8fdce-e5d1-11ee-8f27-128be50dffcf/results'},
    {'href': 'https://ades.test.eodhp.eco-ke-staging.com/temp/water-bodies-e2f8fdce-e5d1-11ee-8f27-128be50dffcf/node_crop.log',
     'title': 'Tool log 

### Undeploy/Delete process

In [17]:
## Here a 204 response means the process was remove successfully, no other content is returned
url = f"https://{ades_endpoint}/{user}/ogc-api/processes/{process_name}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('DELETE', url, headers=headers)
response.status


204